In [1]:
import pandas as pd
a = pd.read_csv('aisdk-2023-04-29.csv')
d = pd.read_csv('aisdk-2023-04-30.csv')
f = pd.read_csv('aisdk-2023-05-01.csv')
dd = a.append(d, ignore_index=True)
df = dd.append(f, ignore_index=True)

In [4]:
from folium import plugins
import geopy.distance
import folium

vessels = [366479000, 219000161, 219000126]
zoom = 7
nyborg = ['55.31274', '10.78964']
colors = iter(['black', 'red', 'purple', 'green', 'blue', 'yellow', 'white'])
try:
    map=folium.Map(location=nyborg, zoom_start=zoom) # tiles='CartoDB positron'
    
    for mmsi in vessels:
        color = next(colors)
        vessel = df.loc[df['MMSI'] == mmsi]
        coords = vessel[['Latitude', 'Longitude']].to_numpy()
        coords = [tuple(x) for x in coords if 0.0 not in x]
        km = 0
        for x in range(len(coords)- 1): km += geopy.distance.geodesic(coords[x], coords[x+1]).km

        folium.PolyLine(coords, tooltip=f'Ship: {mmsi}\nDistance: {round(km, 2)} km', color=color).add_to(map)
        folium.RegularPolygonMarker(location=coords[-1], color=color, number_of_sides=3, radius=5, tooltip=f'Ship: {mmsi}\nDistance: {round(km, 2)} km').add_to(map)
        folium.RegularPolygonMarker(location=coords[0], color=color, number_of_sides=4, radius=5, tooltip=f'Ship: {mmsi}\nDistance: {round(km, 2)} km').add_to(map)
    
    minimap = plugins.MiniMap(toggle_display=True)
    map.add_child(minimap)
    plugins.ScrollZoomToggler().add_to(map)
    plugins.Fullscreen(position="toprigt").add_to(map)
except IndexError:
    print('MMSI doesnt exist in the data.')
except ValueError as err:
    print(err)
map